# Topic Modeling

A popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in the corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Libraries

In [39]:
import pandas as pd
import pickle
from gensim import matutils, models
import scipy.sparse

## LDA : 1st attempt

In [49]:
#Import the Document-term matrix of Simon Sinek
tdm = pd.read_pickle("sinekDtm.pkl")

#Let's put aside the content of simon sinek
sinekCorpus = tdm.Content

#We don't need the content, only the dtm
tdm.drop("Content",axis=1,inplace=True)

In [50]:
tdm.head()

,#Reactions,#Comments,ability,abitofoptimism,able,abnormal,absolutely,accept,accepting,accepts,...,years,yesterday,yin,york,youll,youre,youtube,youtubecomsimonsinek,youve,única
0,12093,257,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5415,164,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,706,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5934,310,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,45732,1200,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
#We remove Reactions & Comments from the Dtm to be used by the algorithm
tdmNoReactions = tdm.drop(['#Reactions','#Comments'],axis=1)

In [52]:
#We transpose the data for gensim
tdmNoReactions = tdmNoReactions.transpose()
tdmNoReactions

,0,1,2,3,4,5,6,7,8,9,...,235,236,237,238,239,240,241,242,243,244
ability,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abitofoptimism,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
able,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abnormal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
absolutely,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
youre,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
youtube,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
youtubecomsimonsinek,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
youve,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts) #It's actually a dtm

In [54]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [55]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.023*"balance" + 0.021*"applecosimonsinek" + 0.021*"accepts" + 0.020*"authorities" + 0.019*"clearer" + 0.019*"actually" + 0.017*"allstart" + 0.017*"civil" + 0.016*"bbc" + 0.015*"animals"'),
 (1,
  '0.033*"applecosimonsinek" + 0.022*"accepts" + 0.020*"balance" + 0.020*"choosing" + 0.019*"actions" + 0.018*"actually" + 0.018*"authorities" + 0.018*"bbc" + 0.017*"aspects" + 0.016*"asked"'),
 (2,
  '0.037*"applecosimonsinek" + 0.028*"accepts" + 0.024*"actually" + 0.020*"clearer" + 0.019*"balance" + 0.018*"authorities" + 0.017*"authentic" + 0.017*"actions" + 0.017*"advancing" + 0.017*"afford"'),
 (3,
  '0.030*"accepts" + 0.025*"applecosimonsinek" + 0.024*"actually" + 0.021*"balance" + 0.021*"actions" + 0.019*"authorities" + 0.017*"clearer" + 0.017*"asked" + 0.016*"action" + 0.016*"advancing"')]

We see some words are unconvenient for this analyse (applecosimonsinek, actually, asked).  
Therefore we can clean the data in a better way.  

For instance we can :
* keep nouns & adjectives only
* remove the words most used

## 2nd attempt

In [57]:
#Let's use content corpus of Simon Sinek's posts we created above
sinekCorpus

0      we are only in charge when we are willing to l...
1      when the people have to manage dangers from in...
2      where do our opinions come from could our assu...
3      if you have an infinite mindset but your leade...
4      if you want to feel happy do something for you...
                             ...                        
240    igniter of the year  well i know that im an op...
241    executives who prioritize the shareholder are ...
242    like many i too have been reflecting as we nea...
243    if you say customer first that means your empl...
244    the small work hard to serve themselves in a b...
Name: Content, Length: 245, dtype: object

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [58]:
#Let's keep nouns & adjectives only
#Let's create a function to pull out nouns & adjectives from a string of text

from nltk import word_tokenize, pos_tag

def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [64]:
#We transform our list of content with the function we have just created
sinekCorpusNounsAdj = sinekCorpus.apply(nouns_adj)
sinekCorpusNounsAdj

0                           charge willing others charge
1      people dangers organization organization able ...
2      opinions assumptions incorrect assumptions civ...
3                       infinite mindset leadership ones
4            happy something fulfilled something someone
                             ...                        
240    igniter year optimist im idealist i world diff...
241    executives shareholder coach desires fair weat...
242    many end year years golden circle origin i sha...
243               customer employees least second people
244          small work big way big work world small way
Name: Content, Length: 245, dtype: object

In [67]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

# Recreate a document-term matrix with only nouns
cvna = CountVectorizer(stop_words=stop_words)
data_cvna = cvna.fit_transform(sinekCorpusNounsAdj)
sinekDtmNounsAdj = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names())
sinekDtmNounsAdj

C:\Users\Jeremy\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ability,abitofoptimism,able,abnormal,accept,act,action,actions,activist,adam,...,yang,year,years,yesterday,yin,york,youll,youtube,youtubecomsimonsinek,única
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
243,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(sinekDtmNounsAdj.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [75]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.014*"optimism" + 0.012*"applecosimonsinek" + 0.012*"bit" + 0.012*"life" + 0.011*"podcasts" + 0.009*"leaders" + 0.009*"world" + 0.008*"team" + 0.007*"new" + 0.007*"work"'),
 (1,
  '0.024*"et" + 0.023*"infinite" + 0.020*"book" + 0.018*"game" + 0.018*"live" + 0.016*"club" + 0.015*"friday" + 0.014*"questions" + 0.012*"week" + 0.008*"linkedin"')]

In [76]:
#3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.014*"live" + 0.014*"et" + 0.014*"optimism" + 0.012*"online" + 0.012*"applecosimonsinek" + 0.011*"life" + 0.011*"bit" + 0.011*"classes" + 0.010*"week" + 0.010*"infinite"'),
 (1,
  '0.015*"book" + 0.014*"friday" + 0.013*"infinite" + 0.013*"et" + 0.013*"live" + 0.011*"game" + 0.011*"club" + 0.010*"leaders" + 0.010*"youtube" + 0.010*"linkedin"'),
 (2,
  '0.018*"book" + 0.016*"club" + 0.014*"team" + 0.014*"questions" + 0.012*"week" + 0.012*"game" + 0.012*"infinite" + 0.011*"bookclubwithsimon" + 0.010*"start" + 0.010*"thank"')]

In [77]:
#4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.030*"et" + 0.017*"week" + 0.017*"book" + 0.017*"questions" + 0.017*"friday" + 0.017*"live" + 0.015*"club" + 0.011*"bookclubwithsimon" + 0.010*"leaders" + 0.009*"online"'),
 (1,
  '0.021*"optimism" + 0.018*"podcasts" + 0.017*"bit" + 0.017*"applecosimonsinek" + 0.012*"way" + 0.009*"good" + 0.008*"truth" + 0.008*"episode" + 0.007*"life" + 0.007*"true"'),
 (2,
  '0.032*"infinite" + 0.026*"game" + 0.021*"live" + 0.015*"book" + 0.013*"linkedin" + 0.012*"chapters" + 0.011*"club" + 0.011*"youtube" + 0.010*"life" + 0.010*"et"'),
 (3,
  '0.019*"optimism" + 0.013*"bit" + 0.011*"podcast" + 0.010*"courage" + 0.010*"leaders" + 0.009*"world" + 0.008*"applecosimonsinek" + 0.008*"season" + 0.007*"value" + 0.007*"great"')]